In [1]:

import time

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from dask_ml.datasets import make_classification

from dpyacl.core.stop_criteria import MaxIteration
from dpyacl.experiment.context import HoldOutExperiment
from dpyacl.metrics import Accuracy
from dpyacl.metrics.evaluation import F1, HammingLoss
from dpyacl.oracle import SimulatedOracle
from dpyacl.scenario.scenario import PoolBasedSamplingScenario
from dpyacl.strategies.single_label import QueryMarginSampling


In [2]:

from dask.distributed import Client
client = Client('tcp://192.168.2.100:8786')
client

Client Scheduler: tcp://192.168.2.100:8786 Dashboard: http://192.168.2.100:8787/status,Cluster Workers: 3 Cores: 6 Memory: 30.00 GB


In [4]:
import dask.array as da
X,y=load_breast_cancer(return_X_y=True) #import the dataset
da.from_array(X)


dask.array<array, shape=(569, 30), dtype=float64, chunksize=(569, 30), chunktype=numpy.ndarray>

In [12]:
iterations = []
for i in range (0,30) :
    X,y=load_breast_cancer(return_X_y=True) #import the dataset

    ml_technique = LogisticRegression(solver='sag')
    stopping_criteria = MaxIteration(25)
    query_strategy = QueryMarginSampling()
    performance_metrics = [Accuracy(),  F1(average='weighted'), HammingLoss()]

    # init the ALExperiment
    experiment = HoldOutExperiment(
        client,
        X,
        y,
        scenario_type=PoolBasedSamplingScenario,
        ml_technique=ml_technique,
        performance_metrics=performance_metrics,
        query_strategy=query_strategy,
        oracle=SimulatedOracle(labels=y),
        stopping_criteria=stopping_criteria,
        self_partition=True,
        test_ratio=0.3,
        initial_label_rate=0.05,
        all_class=False,
        rebalance=True,
        batch_size=1
    )


    start_time = time.time()
    result = experiment.evaluate(client=client, multithread= True, verbose=True)
    end_time = time.time() - start_time
    iterations.append(end_time)

print(iterations)

Label: [1], Cost: [1]

| round | initially labeled data | number of queries | cost | accuracy_score: |  f1_score:   | hamming_loss: |
|   0   |    20 (5.03% of all)   |         1         |  0   |   0.848 ± 0.00  | 0.849 ± 0.00 |  0.152 ± 0.00 |Label: [1], Cost: [1]
|   0   |    20 (5.03% of all)   |         2         |  0   |   0.868 ± 0.02  | 0.869 ± 0.02 |  0.132 ± 0.02 |Label: [1], Cost: [1]
|   0   |    20 (5.03% of all)   |         3         |  0   |   0.879 ± 0.02  | 0.880 ± 0.02 |  0.121 ± 0.02 |Label: [1], Cost: [1]
|   0   |    20 (5.03% of all)   |         4         |  0   |   0.886 ± 0.02  | 0.886 ± 0.02 |  0.114 ± 0.02 |Label: [0], Cost: [1]
|   0   |    20 (5.03% of all)   |         5         |  0   |   0.892 ± 0.02  | 0.892 ± 0.02 |  0.108 ± 0.02 |Label: [1], Cost: [1]
|   0   |    20 (5.03% of all)   |         6         |  0   |   0.895 ± 0.02  | 0.894 ± 0.02 |  0.105 ± 0.02 |Label: [0], Cost: [1]
|   0   |    20 (5.03% of all)   |         7         |  0   |   0.897 ± 0.

In [ ]:
iterations = []
for i in range (0,30) :

    split_count = 10
    feature_num = 30
    label_num = 2
    instance_num = 12000


    X, y = make_classification(n_samples=instance_num, n_features=feature_num,
                           n_informative=2 * label_num,
                           n_redundant=label_num,
                           n_repeated=0,
                           n_classes=label_num,
                           n_clusters_per_class=label_num,
                           weights=None,
                           flip_y=0.01,
                           class_sep=1.0,
                           hypercube=True,
                           shift=0.0,
                           scale=1.0,
                           shuffle=True,
                           random_state=None,
                           chunks=instance_num * 0.10)

    ml_technique = LogisticRegression(solver='sag')
    stopping_criteria = MaxIteration(25)
    query_strategy = QueryMarginSampling()
    performance_metrics = [Accuracy(),  F1(average='weighted'), HammingLoss()]

    # init the ALExperiment
    experiment = HoldOutExperiment(
        client,
        X,
        y,
        scenario_type=PoolBasedSamplingScenario,
        ml_technique=ml_technique,
        performance_metrics=performance_metrics,
        query_strategy=query_strategy,
        oracle=SimulatedOracle(labels=y),
        stopping_criteria=stopping_criteria,
        self_partition=True,
        test_ratio=0.3,
        initial_label_rate=0.05,
        all_class=False,
        rebalance=True,
        batch_size=1
    )


    start_time = time.time()
    result = experiment.evaluate(client=client, multithread= True, verbose=True)
    end_time = time.time() - start_time
    iterations.append(end_time)

print(iterations)